In [12]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import numpy as np
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os


In [13]:
cleandf = pd.read_csv('/Users/anniedbr/Desktop/CSV/data-modesto20170322-131051.csv')

cleandf.head().T


,0,1,2,3,4
pid,6023610939,6023614883,6015724024,6017847804,6017279149
dt,2017-03-22 12:30,2017-03-22 12:30,2017-03-22 12:29,2017-03-22 12:29,2017-03-22 12:29
url,http://modesto.craigslist.org/roo/6023610939.html,http://modesto.craigslist.org/roo/6023614883.html,http://modesto.craigslist.org/roo/6015724024.html,http://modesto.craigslist.org/roo/6017847804.html,http://modesto.craigslist.org/roo/6017279149.html
title,Beautiful single villa rental!,Beautiful single villa rental!,Beautiful single villa rental!,Beautiful single villa rental!,Beautiful single villa rental!
price,NaN,NaN,NaN,600,600
neighb,MODESTO,MODESTO,MODESTO,MODESTO,MODESTO
sqft,0,0,0,460,460
lat,37.6746,37.6746,37.6795,37.6717,37.6772
lng,-121.011,-121.011,-120.944,-120.948,-120.949
accuracy,22,22,7,22,7


In [30]:
OUT_DIR = "/Users/anniedbr/Desktop/Output/"
FNAME_BASE = 'data'

class Cleaning(object):
    #IN_DIR ="C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\scraper_output\\" #James's directory
    #IN_DIR ="C:\\Users\\varun\\Documents\\Berkeley\\2017 Spring\\Workshop\\Datasets\\data_output\\" #Varun's directory
    
    def __init__(self, out_dir = OUT_DIR, fname_base = FNAME_BASE):
        self.out_dir = out_dir
        self.fname_base = fname_base


    def _get_fips(self, row):
        #url = 'http://data.fcc.gov/api/block/find?format=json&latitude={}&longitude={}'
        url = 'http://data.fcc.gov/api/block/find?latitude=[latitude]&longitude=[longitude]&showall=[true/false]'
        request = url.format(row['latitude'], row['longitude'])
        response = requests.get(request)
        data = response.json()
        return pd.Series({'fips_block':data['Block']['FIPS'], 'state':data['State']['code'], 'county':data['County']['name']})
  
    def run(self):
        colnames = ['fips_block','state']
        fname = 'clean' + self.out_dir + self.fname_base + '.csv'
        #directory = os.path.join("/Users/anniedbr/Desktop/CSV/","path")
        #for root,dirs,files in os.walk(directory):
            #for file in files:
                #if file.endswith(".csv"):
        with open (fname, 'wb') as f:
                writer = csv.writer(f)
                writer.writerow(colnames)
                        
                        #for item in 
                        #item_url = domain.split('/search/roo')[0] + row[2]
                        #row[2] = item_url
                        #row += self._get_fips(item_url)
                        #writer.writerow(row) 
                        
        return
        

In [31]:
clean = Cleaning()
clean.run()

FileNotFoundError: [Errno 2] No such file or directory: 'clean/Users/anniedbr/Desktop/Output/data.csv'

In [ ]:

        # TO DO: exception handling for fips
        fips = data_output.apply(self._get_fips, axis=1)             
        geocoded = pd.concat([data_output, fips], axis=1)

        # print('{0} geocoded listings'.format(len(geocoded)))
        return geocoded, len(all_listings), len(thorough_listings), len(geocoded)
   

In [ ]:


    def _clean_listings(self, filename):

        converters = {'neighb':str, 
              'title':str, 
              'price':self._toFloat, 
              'beds':self._toFloat,
              'baths':self._toFloat, 
              'pid':str, 
              'dt':str, 
              'url':str, 
              'sqft':self._toFloat, 
              'sourcepage':str, 
              'lng':self._toFloat, 
              'lat':self._toFloat}

        all_listings = pd.read_csv(filename, converters=converters)

        if len(all_listings) == 0:
            return [], 0, 0, 0
        # print('{0} total listings'.format(len(all_listings)))
        all_listings = all_listings.rename(columns={'price':'rent', 'dt':'date', 'beds':'bedrooms', 'neighb':'neighborhood',
                                                    'baths':'bathrooms','lng':'longitude', 'lat':'latitude'})
        all_listings['rent_sqft'] = all_listings['rent'] / all_listings['sqft']
        all_listings['date'] = pd.to_datetime(all_listings['date'], format='%Y-%m-%d')
        all_listings['day_of_week'] = all_listings['date'].apply(lambda x: x.weekday())
        all_listings['region'] = all_listings['url'].str.extract('http://(.*).craigslist.org', expand=False)
        unique_listings = pd.DataFrame(all_listings.drop_duplicates(subset='pid', inplace=False))
        thorough_listings = pd.DataFrame(unique_listings)
        thorough_listings = thorough_listings[thorough_listings['rent'] > 0]
        thorough_listings = thorough_listings[thorough_listings['sqft'] > 0]
        if len(thorough_listings) == 0:
            return [], 0, 0, 0

        # print('{0} thorough listings'.format(len(thorough_listings)))
        geolocated_filtered_listings = pd.DataFrame(thorough_listings)
        geolocated_filtered_listings = geolocated_filtered_listings[pd.notnull(geolocated_filtered_listings['latitude'])]
        geolocated_filtered_listings = geolocated_filtered_listings[pd.notnull(geolocated_filtered_listings['longitude'])]
        cols = ['pid', 'date', 'region', 'neighborhood', 'rent', 'bedrooms', 'sqft', 'rent_sqft', 'bathrooms', 
                'longitude', 'latitude']
        data_output = geolocated_filtered_listings[cols]

        # TO DO: exception handling for fips
        fips = data_output.apply(self._get_fips, axis=1)             
        geocoded = pd.concat([data_output, fips], axis=1)

        # print('{0} geocoded listings'.format(len(geocoded)))
        return geocoded, len(all_listings), len(thorough_listings), len(geocoded)

